In [1]:
import os
import re
import csv
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from collections import Counter
from PIL import Image
import numpy as np
import random

from tqdm import tqdm

In [2]:
EMBED_DIM = 256
HIDDEN_DIM = 512
LEARNING_RATE = 0.001
BATCH_SIZE = 64
EPOCHS = 50
MIN_WORD_FREQ = 1
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 4

IMAGES_DIR = "flickr8k/Images"
TOKENS_FILE = "flickr8k/captions.txt"

BEST_CHECKPOINT_PATH = "best_checkpoint.pth"  # Checkpoint w/ epoch, model, optimizer, best_val_loss
FINAL_MODEL_PATH = "final_model.pth"          # Final model weights only (saved at the end)
VOCAB_PATH = "vocab.pkl"                      # Where we save the vocabulary

# Set this to True if you want to resume from the best checkpoint
RESUME = False

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
class Vocabulary:
    def __init__(self, freq_threshold=5):
        self.freq_threshold = freq_threshold
        # self.itos = {0: "<pad>", 1: "<start>", 2: "<end>", 3: "<unk>"}
        self.itos = {0: "pad", 1: "startofseq", 2: "endofseq", 3: "unk"}
        self.stoi = {v: k for k, v in self.itos.items()}
        self.index = 4

    def __len__(self):
        return len(self.itos)

    def tokenizer(self, text):
        text = text.lower()
        tokens = re.findall(r"\w+", text)
        return tokens

    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        for sentence in sentence_list:
            tokens = self.tokenizer(sentence)
            frequencies.update(tokens)

        for word, freq in frequencies.items():
            if freq >= self.freq_threshold:
                self.stoi[word] = self.index
                self.itos[self.index] = word
                self.index += 1

    def numericalize(self, text):
        tokens = self.tokenizer(text)
        numericalized = []
        for token in tokens:
            if token in self.stoi:
                numericalized.append(self.stoi[token])
            else:
                numericalized.append(self.stoi["<unk>"])
        return numericalized

In [4]:
def parse_flickr_tokens(csv_file):
    """
    Reads a CSV file with columns: image,caption
    Returns dict: {image_filename: [captions]}
    """
    imgid2captions = {}
    with open(csv_file, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        # Skip header row: "image,caption"
        next(reader, None)
        for row in reader:
            if len(row) < 2:
                continue
            img_id, caption = row[0], row[1]
            if img_id not in imgid2captions:
                imgid2captions[img_id] = []
            imgid2captions[img_id].append(caption)
    return imgid2captions

class Flickr8kDataset(Dataset):
    def __init__(self, imgid2captions, vocab, transform=None):
        self.imgid2captions = []
        self.transform = transform
        self.vocab = vocab

        # Flatten each (img_id, [cap1, cap2, ...]) into multiple examples
        for img_id, caps in imgid2captions.items():
            for c in caps:
                self.imgid2captions.append((img_id, c))

    def __len__(self):
        return len(self.imgid2captions)

    def __getitem__(self, idx):
        img_id, caption = self.imgid2captions[idx]
        # print('CAPTION START...', caption, 'CAPTION END\n')
        img_path = os.path.join(IMAGES_DIR, img_id)

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # with open('val_file.txt', 'a') as f:
        #     f.writelines(f"{img_path}: {caption}\n")

        # Numericalize caption
        # numerical_caption = [self.vocab.stoi["<start>"]]
        numerical_caption = [self.vocab.stoi["startofseq"]]
        numerical_caption += self.vocab.numericalize(caption)
        numerical_caption.append(self.vocab.stoi["endofseq"])

        return image, torch.tensor(numerical_caption, dtype=torch.long)

In [5]:
def collate_fn(batch):
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    images = [item[0] for item in batch]
    captions = [item[1] for item in batch]
    lengths = [len(cap) for cap in captions]
    max_len = max(lengths)

    padded_captions = torch.zeros(len(captions), max_len, dtype=torch.long)
    for i, cap in enumerate(captions):
        end = lengths[i]
        padded_captions[i, :end] = cap[:end]

    images = torch.stack(images, dim=0)
    return images, padded_captions, lengths

In [6]:
class ResNetEncoder(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        for param in resnet.parameters():
            param.requires_grad = True
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        
        self.fc = nn.Linear(resnet.fc.in_features, embed_dim)
        self.batch_norm = nn.BatchNorm1d(embed_dim, momentum=0.01)

    def forward(self, images):
        with torch.no_grad():
            features = self.resnet(images)  # (batch_size, 2048, 1, 1)
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        features = self.batch_norm(features)
        return features

class DecoderLSTM(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, features, captions):
        # remove the last token for input
        captions_in = captions[:, :-1]
        emb = self.embedding(captions_in)
        features = features.unsqueeze(1)
        lstm_input = torch.cat((features, emb), dim=1)
        outputs, _ = self.lstm(lstm_input)
        logits = self.fc(outputs)
        return logits

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs

In [7]:
def train_one_epoch(model, dataloader, criterion, optimizer, vocab_size, epoch):
    model.train()
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}", unit="batch")
    for images, captions, _lengths in progress_bar:
        images = images.to(DEVICE)
        captions = captions.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images, captions)
        # outputs: (batch_size, seq_len, vocab_size)
        # compare outputs[:, 1:, :] with captions[:, 1:]
        outputs = outputs[:, 1:, :].contiguous().view(-1, vocab_size)
        targets = captions[:, 1:].contiguous().view(-1)

        # outputs = outputs.contiguous().view(-1, vocab_size)
        # targets = captions.contiguous().view(-1)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
    avg_loss = total_loss / len(dataloader)
    return avg_loss

def validate(model, dataloader, criterion, vocab_size):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, captions, _lengths in dataloader:
            images = images.to(DEVICE)
            captions = captions.to(DEVICE)
            outputs = model(images, captions)
            outputs = outputs[:, 1:, :].contiguous().view(-1, vocab_size)
            targets = captions[:, 1:].contiguous().view(-1)
            
            # outputs = outputs.contiguous().view(-1, vocab_size)
            # targets = captions.contiguous().view(-1)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    avg_val_loss = total_loss / len(dataloader)
    return avg_val_loss

In [8]:
print(f"Using device: {DEVICE}")

# ---------------------------------------
# (A) Parse tokens and build vocabulary
# ---------------------------------------
if not RESUME:
    # If not resuming, parse and build vocab from scratch, and create pkl
    imgid2captions = parse_flickr_tokens(TOKENS_FILE)

    all_captions = []
    for caps in imgid2captions.values():
        all_captions.extend(caps)

    vocab = Vocabulary(freq_threshold=MIN_WORD_FREQ)
    vocab.build_vocabulary(all_captions)

    with open(VOCAB_PATH, "wb") as f:
        pickle.dump(vocab, f)
    print("Vocabulary saved to:", VOCAB_PATH)

    vocab_size = len(vocab)
    print(f"Vocabulary size: {vocab_size}")

    img_ids = list(imgid2captions.keys())
    random.shuffle(img_ids)
    split_idx = int(0.8 * len(img_ids))
    train_ids = img_ids[:split_idx]
    val_ids = img_ids[split_idx:]

    train_dict = {iid: imgid2captions[iid] for iid in train_ids}
    val_dict = {iid: imgid2captions[iid] for iid in val_ids}

else:
    # If resuming, we assume vocab has been built already, so load it
    with open(VOCAB_PATH, "rb") as f:
        vocab = pickle.load(f)
    vocab_size = len(vocab)
    print(f"Resuming training. Vocab size: {vocab_size}")

    # Also, parse the tokens again
    imgid2captions = parse_flickr_tokens(TOKENS_FILE)
    # or you can store train/val splits in a file if you'd like, but let's do it again
    img_ids = list(imgid2captions.keys())
    random.shuffle(img_ids)
    split_idx = int(0.8 * len(img_ids))
    train_ids = img_ids[:split_idx]
    val_ids = img_ids[split_idx:]

    train_dict = {iid: imgid2captions[iid] for iid in train_ids}
    val_dict = {iid: imgid2captions[iid] for iid in val_ids}

# ---------------------------------------
# (B) Create datasets & loaders
# ---------------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

train_dataset = Flickr8kDataset(train_dict, vocab, transform=transform)
val_dataset = Flickr8kDataset(val_dict, vocab, transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=False,
    num_workers=NUM_WORKERS
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    drop_last=False,
    num_workers=NUM_WORKERS
)

# ---------------------------------------
# (C) Create model, optimizer, etc.
# ---------------------------------------
encoder = ResNetEncoder(EMBED_DIM)
decoder = DecoderLSTM(EMBED_DIM, HIDDEN_DIM, vocab_size)
model = ImageCaptioningModel(encoder, decoder).to(DEVICE)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

# criterion = nn.CrossEntropyLoss(ignore_index=vocab.stoi["<pad>"])
criterion = nn.CrossEntropyLoss(ignore_index=vocab.stoi["pad"])
parameters = list(model.decoder.parameters()) + list(model.encoder.fc.parameters()) + list(model.encoder.batch_norm.parameters())
optimizer = optim.Adam(parameters, lr=LEARNING_RATE)

start_epoch = 0
best_val_loss = float("inf")

# If we want to resume from an existing checkpoint
if RESUME and os.path.exists(BEST_CHECKPOINT_PATH):
    print("Resuming from checkpoint:", BEST_CHECKPOINT_PATH)
    checkpoint = torch.load(BEST_CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    best_val_loss = checkpoint["best_val_loss"]
    print(f"Resuming at epoch {start_epoch}, best_val_loss so far: {best_val_loss:.4f}")
elif RESUME:
    print(f"Warning: {BEST_CHECKPOINT_PATH} not found. Starting fresh...")

# ---------------------------------------
# (D) Training Loop
# ---------------------------------------
try:
    for epoch in range(start_epoch, EPOCHS):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, vocab_size, epoch)
        val_loss = validate(model, val_loader, criterion, vocab_size)

        print(f"[Epoch {epoch+1}/{EPOCHS}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Save checkpoint if it's the best so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            checkpoint_dict = {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_val_loss": best_val_loss
            }
            torch.save(checkpoint_dict, BEST_CHECKPOINT_PATH)
            print(f"New best model saved -> {BEST_CHECKPOINT_PATH} (val_loss={val_loss:.4f})")

        final_checkpoint_dict = {
                "model_state_dict": model.state_dict(),
            }
        torch.save(final_checkpoint_dict, FINAL_MODEL_PATH)

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Best checkpoint is already saved if it improved during training.")

print(f"\nFinal model weights saved to {FINAL_MODEL_PATH}")
print(f"Best val_loss={best_val_loss:.4f} (checkpoint at {BEST_CHECKPOINT_PATH})")

Using device: cuda
Vocabulary saved to: vocab.pkl
Vocabulary size: 8492
32,140,396 total parameters.
32,140,396 training parameters.


Epoch 1: 100%|████████████████| 506/506 [01:03<00:00,  8.03batch/s, loss=1.9518]


[Epoch 1/50] Train Loss: 2.1531 | Val Loss: 1.7488
New best model saved -> best_checkpoint.pth (val_loss=1.7488)


Epoch 2: 100%|████████████████| 506/506 [01:12<00:00,  6.96batch/s, loss=1.5735]


[Epoch 2/50] Train Loss: 1.5907 | Val Loss: 1.6098
New best model saved -> best_checkpoint.pth (val_loss=1.6098)


...

[Epoch 48/50] Train Loss: 0.1768 | Val Loss: 2.7922


Epoch 49: 100%|███████████████| 506/506 [01:04<00:00,  7.84batch/s, loss=0.1990]


[Epoch 49/50] Train Loss: 0.1756 | Val Loss: 2.8080


Epoch 50: 100%|███████████████| 506/506 [01:04<00:00,  7.83batch/s, loss=0.2131]


[Epoch 50/50] Train Loss: 0.1747 | Val Loss: 2.8259

Final model weights saved to final_model.pth
Best val_loss=1.5548 (checkpoint at best_checkpoint.pth)
